<a href="https://colab.research.google.com/github/MartianEagle007/ML/blob/main/category_classification_BBC_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
#tf.enable_eager_execution()
import os
import datetime
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('bbc-text.csv')

In [ ]:
df.columns

Index(['cat', 'description'], dtype='object')

In [ ]:
X_train,X_test=train_test_split(df,test_size=0.2,random_state=111)

In [ ]:
from sklearn.utils import class_weight
class_weight =list(class_weight.compute_class_weight('balanced',np.unique(df['cat']),df['cat']))

In [ ]:
df['cat'].value_counts()

business         510
politics         417
entertainment    386
sport              4
tech               2
Name: cat, dtype: int64

In [ ]:
class_weight.sort()

In [ ]:
class_weight

[0.5172549019607843, 0.6326139088729017, 0.683419689119171, 65.95, 131.9]

In [ ]:
weights={}

In [ ]:
for index, weight in enumerate(class_weight):
    weights[index]=weight

In [ ]:
weights

{0: 0.5172549019607843,
 1: 0.6326139088729017,
 2: 0.683419689119171,
 3: 65.95,
 4: 131.9}

In [ ]:
dataset_train=tf.data.Dataset.from_tensor_slices((X_train['description'].values,X_train['cat'].values))
dataset_test=tf.data.Dataset.from_tensor_slices((X_test['description'].values,X_test['cat'].values))

In [ ]:
for text, target in dataset_train.take(4):
  print('description: {},Target: {}'.format(text,target))

description: b'labour chooses manchester the labour party will hold its 2006 autumn conference in manchester and not blackpool  it has been confirmed.  the much trailed decision was ratified by labour s ruling national executive committee in a break with the traditional choice of a seaside venue. it will be the first time since 1917 that the party has chosen manchester to host the annual event. blackpool will get the much smaller february spring conference instead in what will be seen as a placatory move.  for years the main political parties have rotated between blackpool  bournemouth and brighton. and the news the much larger annual conference is not to gather in blackpool will be seen as a blow in the coastal resort. in 1998 the party said it would not return to blackpool but did so in 2002. the following year bournemouth hosted the event before the party signed a two year deal for brighton to host the autumn conference.  colin asplin  blackpool hotel association said:  we have trie

In [ ]:
for text, target in dataset_test.take(4):
  print('description: {},Target: {}'.format(text,target))

description: b'uk troops on ivory coast standby downing street has confirmed british troops are on standby in case they need to help evacuate several hundred uk citizens from ivory coast.  the news came as it emerged france had begun evacuating its hostages after days of anti-french demonstrations. trouble flared after nine french peacekeepers were killed and president jacques chirac ordered the destruction of the ivory coast s air force. a company of 100-120 uk troops is understood to have been put on standby. they are ready to fly out and secure the route from the embassy to the airport if called upon. prime minister tony blair s official spokesman said:  there are british nationals in ivory coast and as you can imagine we are making the usual contingency plans in case the situation deteriorates further  but that s all i can say at this stage.   he added the situation would be monitored carefully in case evacuation became necessary. france has played a lead role in international peac

In [ ]:
table=tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['business','politics','entertainment','sport','tech' ]),
        values=tf.constant([0,1,2,3,4]),
    ),
    default_value=tf.constant(-1),
    name="target_encoding"
)

@tf.function
def target(x):
    return table.lookup(x)

In [ ]:
def show_batch(dataset,size=5):
              for batch,label in dataset.take(size):
                  print(batch.numpy())
                  print(target(label).numpy())

In [ ]:
show_batch(dataset_test,5)

b'uk troops on ivory coast standby downing street has confirmed british troops are on standby in case they need to help evacuate several hundred uk citizens from ivory coast.  the news came as it emerged france had begun evacuating its hostages after days of anti-french demonstrations. trouble flared after nine french peacekeepers were killed and president jacques chirac ordered the destruction of the ivory coast s air force. a company of 100-120 uk troops is understood to have been put on standby. they are ready to fly out and secure the route from the embassy to the airport if called upon. prime minister tony blair s official spokesman said:  there are british nationals in ivory coast and as you can imagine we are making the usual contingency plans in case the situation deteriorates further  but that s all i can say at this stage.   he added the situation would be monitored carefully in case evacuation became necessary. france has played a lead role in international peacekeeping in i

In [ ]:
def fetch(text,labels):
    return text, tf.one_hot(target(labels),5)

In [ ]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [ ]:
next(iter(train_data_f))

(<tf.Tensor: shape=(), dtype=string, numpy=b'labour chooses manchester the labour party will hold its 2006 autumn conference in manchester and not blackpool  it has been confirmed.  the much trailed decision was ratified by labour s ruling national executive committee in a break with the traditional choice of a seaside venue. it will be the first time since 1917 that the party has chosen manchester to host the annual event. blackpool will get the much smaller february spring conference instead in what will be seen as a placatory move.  for years the main political parties have rotated between blackpool  bournemouth and brighton. and the news the much larger annual conference is not to gather in blackpool will be seen as a blow in the coastal resort. in 1998 the party said it would not return to blackpool but did so in 2002. the following year bournemouth hosted the event before the party signed a two year deal for brighton to host the autumn conference.  colin asplin  blackpool hotel a

In [ ]:
train_data,train_labels=next(iter(train_data_f.batch(4)))
train_data,train_labels

(<tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'labour chooses manchester the labour party will hold its 2006 autumn conference in manchester and not blackpool  it has been confirmed.  the much trailed decision was ratified by labour s ruling national executive committee in a break with the traditional choice of a seaside venue. it will be the first time since 1917 that the party has chosen manchester to host the annual event. blackpool will get the much smaller february spring conference instead in what will be seen as a placatory move.  for years the main political parties have rotated between blackpool  bournemouth and brighton. and the news the much larger annual conference is not to gather in blackpool will be seen as a blow in the coastal resort. in 1998 the party said it would not return to blackpool but did so in 2002. the following year bournemouth hosted the event before the party signed a two year deal for brighton to host the autumn conference.  colin asplin  blackp

In [ ]:
embedding="https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer=hub.KerasLayer(embedding,output_shape=[128],input_shape=[],
                        dtype=tf.string,trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 1.6941034 ,  0.23049523, -0.08793923,  0.01198543,  0.3070997 ,
        -0.13066319, -0.32739973, -0.31352264, -0.07746296,  0.40974587,
         0.39540884, -0.03749761, -0.14638169, -0.22685687,  0.02108006,
        -0.16677113, -0.23329513, -0.15148026, -0.3090214 ,  1.4332433 ,
         0.06916769,  0.4247905 ,  0.06908006,  0.01716453,  0.24063616,
        -0.10666896,  0.09301689,  0.03350942, -0.3637271 , -0.06710243,
         0.15618017, -0.06435317,  0.2852476 ,  0.18387385,  0.00244654,
         0.05746726, -0.17328995, -0.13956442, -0.0453722 ,  0.44838452,
        -0.16400109,  0.05425557, -0.24944031, -0.05772451,  0.2594732 ,
         0.12612946,  0.03629567,  0.00397563, -0.05138079,  0.01340066,
         0.09114183,  0.18002254, -0.02654433,  0.00932222, -0.37852114,
         0.12668972, -0.07056652, -0.1214018 ,  0.06388934,  0.09388972,
        -0.40544865,  0.15294464,  0.18924394, -0.29335105, -0.12005391,
 

In [ ]:
model=tf.keras.Sequential()
model.add(hub_layer)
for units in [128,128,64,32]:
    model.add(tf.keras.layers.Dense(units,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(5,activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)               

In [ ]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [ ]:
train_data_f=train_data_f.shuffle(50000).batch(512)
test_data_f=test_data_f.batch(512)

In [ ]:
history=model.fit(train_data_f,
                 epochs=10,
                 validation_data=test_data_f,
                 verbose=1,
                 class_weight=weights)

Epoch 1/10
3/3 [==============================] - 3s 915ms/step - loss: 1.4637 - accuracy: 0.4692 - val_loss: 1.4422 - val_accuracy: 0.5038
Epoch 2/10
3/3 [==============================] - 3s 896ms/step - loss: 1.4656 - accuracy: 0.5033 - val_loss: 1.4072 - val_accuracy: 0.5909
Epoch 3/10
3/3 [==============================] - 3s 889ms/step - loss: 1.4369 - accuracy: 0.5336 - val_loss: 1.3766 - val_accuracy: 0.6326
Epoch 4/10
3/3 [==============================] - 3s 911ms/step - loss: 1.3745 - accuracy: 0.5611 - val_loss: 1.3512 - val_accuracy: 0.6326
Epoch 5/10
3/3 [==============================] - 3s 895ms/step - loss: 1.3338 - accuracy: 0.5678 - val_loss: 1.3247 - val_accuracy: 0.6326
Epoch 6/10
3/3 [==============================] - 3s 889ms/step - loss: 1.3012 - accuracy: 0.5848 - val_loss: 1.2880 - val_accuracy: 0.6553
Epoch 7/10
3/3 [==============================] - 3s 895ms/step - loss: 1.2333 - accuracy: 0.6284 - val_loss: 1.2547 - val_accuracy: 0.6553
Epoch 8/10
3/3 [====

In [ ]:
len(list(dataset_test))

264

In [ ]:
results=model.evaluate(dataset_test.map(fetch).batch(264),verbose=2)
print(results)

1/1 - 0s - loss: 1.1623 - accuracy: 0.8030
[1.1623077392578125, 0.8030303120613098]


In [ ]:
test_data,test_labels=next(iter(dataset_test.map(fetch).batch(45963)))

In [ ]:
y_pred=model.predict(test_data)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_labels.numpy().argmax(axis=1),y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        98
           1       1.00      0.45      0.62        89
           2       0.73      0.97      0.83        76
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1

    accuracy                           0.80       264
   macro avg       0.52      0.48      0.48       264
weighted avg       0.87      0.80      0.79       264



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix (test_labels.numpy().argmax(axis=1),y_pred.argmax(axis=1))

array([[98,  0,  0,  0,  0],
       [13, 40, 27,  1,  8],
       [ 2,  0, 74,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0]])